# Clustering Crypto

In [109]:
# Initial imports
import pandas as pd
import hvplot.pandas
from path import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


### Deliverable 1: Preprocessing the Data for PCA

In [112]:
# Load the crypto_data.csv dataset.
file_path = "Resources/crypto_data.csv"
crypto_df = pd.read_csv(file_path, index_col=0)
crypto_df.head()

CoinName Algorithm  IsTrading ProofType  TotalCoinsMined TotalCoinSupply
42     42 Coin    Scrypt       True   PoW/PoS     4.199995e+01              42
365    365Coin       X11       True   PoW/PoS              NaN      2300000000
404    404Coin    Scrypt       True   PoW/PoS     1.055185e+09       532000000
611  SixEleven   SHA-256       True       PoW              NaN          611000
808        808   SHA-256       True   PoW/PoS     0.000000e+00               0

In [113]:
crypto_df.dtypes

CoinName            object
Algorithm           object
IsTrading             bool
ProofType           object
TotalCoinsMined    float64
TotalCoinSupply     object
dtype: object

In [114]:
# Keep all the cryptocurrencies that are being traded.
crypto_df= crypto_df[crypto_df['IsTrading'] == True]
crypto_df.count()


CoinName           1144
Algorithm          1144
IsTrading          1144
ProofType          1144
TotalCoinsMined     685
TotalCoinSupply    1144
dtype: int64

In [115]:
# Keep all the cryptocurrencies that have a working algorithm.
crypto_df = crypto_df[crypto_df.Algorithm.isna()==False]
crypto_df.count()

CoinName           1144
Algorithm          1144
IsTrading          1144
ProofType          1144
TotalCoinsMined     685
TotalCoinSupply    1144
dtype: int64

In [116]:
crypto_df.head()

CoinName Algorithm  IsTrading ProofType  TotalCoinsMined TotalCoinSupply
42     42 Coin    Scrypt       True   PoW/PoS     4.199995e+01              42
365    365Coin       X11       True   PoW/PoS              NaN      2300000000
404    404Coin    Scrypt       True   PoW/PoS     1.055185e+09       532000000
611  SixEleven   SHA-256       True       PoW              NaN          611000
808        808   SHA-256       True   PoW/PoS     0.000000e+00               0

In [117]:
# Remove the "IsTrading" column. 
crypto_df.drop(columns=["IsTrading"], inplace=True)
crypto_df.head()

CoinName Algorithm ProofType  TotalCoinsMined TotalCoinSupply
42     42 Coin    Scrypt   PoW/PoS     4.199995e+01              42
365    365Coin       X11   PoW/PoS              NaN      2300000000
404    404Coin    Scrypt   PoW/PoS     1.055185e+09       532000000
611  SixEleven   SHA-256       PoW              NaN          611000
808        808   SHA-256   PoW/PoS     0.000000e+00               0

In [118]:
# Remove rows that have at least 1 null value.
crypto_df = crypto_df.dropna()
crypto_df.head(5)

CoinName Algorithm ProofType  TotalCoinsMined TotalCoinSupply
42      42 Coin    Scrypt   PoW/PoS     4.199995e+01              42
404     404Coin    Scrypt   PoW/PoS     1.055185e+09       532000000
808         808   SHA-256   PoW/PoS     0.000000e+00               0
1337  EliteCoin       X13   PoW/PoS     2.927942e+10    314159265359
BTC     Bitcoin   SHA-256       PoW     1.792718e+07        21000000

In [119]:
print(crypto_df.shape)
print(crypto_df.count())

(685, 5)
CoinName           685
Algorithm          685
ProofType          685
TotalCoinsMined    685
TotalCoinSupply    685
dtype: int64


In [120]:
# Keep the rows where coins are mined.
crypto_df = crypto_df[crypto_df['TotalCoinsMined'] > 0]

In [121]:
print(crypto_df.shape)
print(crypto_df.count())

(532, 5)
CoinName           532
Algorithm          532
ProofType          532
TotalCoinsMined    532
TotalCoinSupply    532
dtype: int64


In [122]:
# Create a new DataFrame that holds only the cryptocurrencies names.
coin_names_df = crypto_df[['CoinName']]
print(coin_names_df.shape)
print(coin_names_df.count())
coin_names_df.head()

(532, 1)
CoinName    532
dtype: int64


CoinName
42      42 Coin
404     404Coin
1337  EliteCoin
BTC     Bitcoin
ETH    Ethereum

In [123]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
crypto_df.drop(columns=["CoinName"], inplace=True)
print(crypto_df.shape)
print(crypto_df.count())
crypto_df.head()

(532, 4)
Algorithm          532
ProofType          532
TotalCoinsMined    532
TotalCoinSupply    532
dtype: int64


Algorithm ProofType  TotalCoinsMined TotalCoinSupply
42      Scrypt   PoW/PoS     4.199995e+01              42
404     Scrypt   PoW/PoS     1.055185e+09       532000000
1337       X13   PoW/PoS     2.927942e+10    314159265359
BTC    SHA-256       PoW     1.792718e+07        21000000
ETH     Ethash       PoW     1.076842e+08               0

In [124]:
crypto_df.head(10)

Algorithm ProofType  TotalCoinsMined TotalCoinSupply
42            Scrypt   PoW/PoS     4.199995e+01              42
404           Scrypt   PoW/PoS     1.055185e+09       532000000
1337             X13   PoW/PoS     2.927942e+10    314159265359
BTC          SHA-256       PoW     1.792718e+07        21000000
ETH           Ethash       PoW     1.076842e+08               0
LTC           Scrypt       PoW     6.303924e+07        84000000
DASH             X11   PoW/PoS     9.031294e+06        22000000
XMR   CryptoNight-V7       PoW     1.720114e+07               0
ETC           Ethash       PoW     1.133597e+08       210000000
ZEC         Equihash       PoW     7.383056e+06        21000000

In [125]:
# Use get_dummies() to create variables for text features.
X = pd.get_dummies(crypto_df, columns=["Algorithm", "ProofType"])
print(X.shape)
X.head()


(532, 98)


TotalCoinsMined TotalCoinSupply  Algorithm_1GB AES Pattern Search  \
42       4.199995e+01              42                                 0   
404      1.055185e+09       532000000                                 0   
1337     2.927942e+10    314159265359                                 0   
BTC      1.792718e+07        21000000                                 0   
ETH      1.076842e+08               0                                 0   

      Algorithm_536  Algorithm_Argon2d  Algorithm_BLAKE256  Algorithm_Blake  \
42                0                  0                   0                0   
404               0                  0                   0                0   
1337              0                  0                   0                0   
BTC               0                  0                   0                0   
ETH               0                  0                   0                0   

      Algorithm_Blake2S  Algorithm_Blake2b  Algorithm_C11  ...  \
42                    0                  0              0  ...   
404                   0                  0              0  ...   
1337                  0                  0              0  ...   
BTC                   0                  0              0  ...   
ETH                   0                  0              0  ...   

      ProofType_PoW/PoS  ProofType_PoW/PoS   ProofType_PoW/PoW  \
42                    1                   0                  0   
404                   1                   0                  0   
1337                  1                   0                  0   
BTC                   0                   0                  0   
ETH                   0                   0                  0   

      ProofType_PoW/nPoS  ProofType_Pos  ProofType_Proof of Authority  \
42                     0              0                             0   
404                    0              0                             0   
1337                   0              0                             0   
BTC                    0              0                             0   
ETH                    0              0                             0   

      ProofType_Proof of Trust  ProofType_TPoS  \
42                           0               0   
404                          0               0   
1337                         0               0   
BTC                          0               0   
ETH                          0               0   

      ProofType_Zero-Knowledge Proof  ProofType_dPoW/PoW  
42                                 0                   0  
404                                0                   0  
1337                               0                   0  
BTC                                0                   0  
ETH                                0                   0  

[5 rows x 98 columns]

In [126]:
# Standardize the data with StandardScaler().
data_scaler = StandardScaler()
X = data_scaler.fit_transform(X)
X[:5]

array([[-0.11710817, -0.1528703 , -0.0433963 , -0.0433963 , -0.0433963 ,
        -0.06142951, -0.07530656, -0.0433963 , -0.06142951, -0.06142951,
        -0.0433963 , -0.0433963 , -0.19245009, -0.06142951, -0.09740465,
        -0.0433963 , -0.11547005, -0.07530656, -0.0433963 , -0.0433963 ,
        -0.15191091, -0.0433963 , -0.13118084, -0.0433963 , -0.0433963 ,
        -0.08703883, -0.0433963 , -0.0433963 , -0.0433963 , -0.0433963 ,
        -0.06142951, -0.0433963 , -0.08703883, -0.08703883, -0.08703883,
        -0.0433963 , -0.13118084, -0.13840913, -0.13840913, -0.0433963 ,
        -0.06142951, -0.0433963 , -0.07530656, -0.18168574, -0.0433963 ,
        -0.0433963 , -0.0433963 , -0.07530656, -0.15826614, -0.31491833,
        -0.0433963 , -0.08703883, -0.07530656, -0.06142951,  1.38675049,
        -0.0433963 , -0.0433963 , -0.06142951, -0.0433963 , -0.0433963 ,
        -0.0433963 , -0.0433963 , -0.0433963 , -0.0433963 , -0.0433963 ,
        -0.0433963 , -0.39879994, -0.0433963 , -0.1

### Deliverable 2: Reducing Data Dimensions Using PCA

In [127]:
# Using PCA to reduce dimension to three principal components.
# Initialize PCA model
pca = PCA(n_components=3)
crypto_pca = pca.fit_transform(X)
crypto_pca

array([[-0.32303152,  0.93454028, -0.44938918],
       [-0.30640331,  0.93491546, -0.44980227],
       [ 2.30232719,  1.62996882, -0.53794299],
       ...,
       [ 0.31019483, -2.2501561 ,  0.33933494],
       [-0.1702913 , -1.94951759,  0.30859561],
       [-0.2982958 ,  0.86160083, -0.23830273]])

In [129]:
# Create a DataFrame with the three principal components.
pcs_df = pd.DataFrame(
    data=crypto_pca, columns=["PC 1","PC 2", "PC 3"], index=crypto_df.index)
pcs_df.head()

PC 1      PC 2      PC 3
42   -0.323032  0.934540 -0.449389
404  -0.306403  0.934915 -0.449802
1337  2.302327  1.629969 -0.537943
BTC  -0.150621 -1.296610  0.153397
ETH  -0.166397 -1.974263  0.324855

### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [130]:
# Create an elbow curve to find the best value for K.

inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of K values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pcs_df)
    inertia.append(km.inertia_)

# Create the elbow curve
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")

C:\Users\aaron\anaconda3\envs\mlenv\lib\site-packages\sklearn\cluster\_kmeans.py:1037: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.
  "KMeans is known to have a memory leak on Windows "


:Curve   [k]   (inertia)

Running K-Means with `k=4`

In [135]:
# Initialize the K-Means model.
model = KMeans(n_clusters=4, random_state=0)

# Fit the model
model.fit(pcs_df)

# Predict clusters
predictions = model.predict(pcs_df)
predictions



array([0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0,
       1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1,
       1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0,
       0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1,
       0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0,
       1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1,
       0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0,

In [136]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
clustered_df = pd.concat([crypto_df, pcs_df], axis=1, join='inner')

#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
clustered_df["CoinName"]=coin_names_df.CoinName

#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
clustered_df["Class"]=model.labels_

# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)

(532, 9)


Algorithm ProofType  TotalCoinsMined TotalCoinSupply      PC 1  \
42            Scrypt   PoW/PoS     4.199995e+01              42 -0.323032   
404           Scrypt   PoW/PoS     1.055185e+09       532000000 -0.306403   
1337             X13   PoW/PoS     2.927942e+10    314159265359  2.302327   
BTC          SHA-256       PoW     1.792718e+07        21000000 -0.150621   
ETH           Ethash       PoW     1.076842e+08               0 -0.166397   
LTC           Scrypt       PoW     6.303924e+07        84000000 -0.166493   
DASH             X11   PoW/PoS     9.031294e+06        22000000 -0.372747   
XMR   CryptoNight-V7       PoW     1.720114e+07               0 -0.151331   
ETC           Ethash       PoW     1.133597e+08       210000000 -0.164845   
ZEC         Equihash       PoW     7.383056e+06        21000000 -0.170290   

          PC 2      PC 3          CoinName  Class  
42    0.934540 -0.449389           42 Coin      0  
404   0.934915 -0.449802           404Coin      0  
1337  1.629969 -0.537943         EliteCoin      0  
BTC  -1.296610  0.153397           Bitcoin      1  
ETH  -1.974263  0.324855          Ethereum      1  
LTC  -1.136364  0.007235          Litecoin      1  
DASH  1.118739 -0.383844              Dash      0  
XMR  -2.238748  0.354822            Monero      1  
ETC  -1.974335  0.324835  Ethereum Classic      1  
ZEC  -1.949518  0.308596             ZCash      1

### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [141]:
# Creating a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(clustered_df, x="PC 1", y="PC 2", z="PC 3", color="Class", symbol="Class", width=800, hover_name="CoinName",
                   hover_data=["Algorithm"])
fig.update_layout(legend=dict(x=0,y=1))
fig.show()


In [142]:
# Create a table with tradable cryptocurrencies.
clustered_df.hvplot.table(columns=['CoinName', 'Algorithm', 'ProofType', 'TotalCoinSupply', 'TotalCoinsMined','Class'], sortable=True, selectable=True)

:Table   [CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,Class]

In [146]:
# Print the total number of tradable cryptocurrencies.
print(f'There are {len(clustered_df.index)} tradable cryptocurrencies')


There are 532 tradable cryptocurrencies


In [149]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
clustered_df_scaled = clustered_df[['TotalCoinSupply','TotalCoinsMined']]
clustered_df_scaled = MinMaxScaler().fit_transform(clustered_df_scaled)
clustered_df_scaled

array([[4.20000000e-11, 0.00000000e+00],
       [5.32000000e-04, 1.06585544e-03],
       [3.14159265e-01, 2.95755135e-02],
       ...,
       [1.40022261e-03, 9.90135079e-04],
       [2.10000000e-05, 7.37028150e-06],
       [1.00000000e-06, 1.29582282e-07]])

In [151]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
plot_df = pd.DataFrame(clustered_df_scaled, columns=['TotalCoinSupply','TotalCoinsMined'],index=clustered_df.index)

# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
plot_df['CoinName'] = clustered_df.CoinName

# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
plot_df['Class'] = clustered_df.Class

plot_df.head(10)

TotalCoinSupply  TotalCoinsMined          CoinName  Class
42       4.200000e-11         0.000000           42 Coin      0
404      5.320000e-04         0.001066           404Coin      0
1337     3.141593e-01         0.029576         EliteCoin      0
BTC      2.100000e-05         0.000018           Bitcoin      1
ETH      0.000000e+00         0.000109          Ethereum      1
LTC      8.400000e-05         0.000064          Litecoin      1
DASH     2.200000e-05         0.000009              Dash      0
XMR      0.000000e+00         0.000017            Monero      1
ETC      2.100000e-04         0.000115  Ethereum Classic      1
ZEC      2.100000e-05         0.000007             ZCash      1

In [158]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
plot_df.hvplot.scatter(x="TotalCoinsMined", y="TotalCoinSupply", by="Class", hover_cols="CoinName")

:NdOverlay   [Class]
   :Scatter   [TotalCoinsMined]   (TotalCoinSupply,CoinName)